In [ ]:
# Import necessary libraries
import os
import sys
import torch
import random
import threading
import numpy as np
import pandas as pd
from pathlib import Path

# Set environment variables for GPU and device configuration
# os.environ['CUDA_VISIBLE_DEVICES'] = '5'  # Uncomment to set a different GPU
%env CUDA_VISIBLE_DEVICES=4
device = torch.device('cuda:4' if torch.cuda.is_available() else "cpu")

# Add custom paths to the Python environment
sys.path.append('./fastai1/')

# Disable RDKit warning messages
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# Import FastAI modules
from fastai import *
from fastai.text import *
from fastai.vision import *
from fastai.imports import *

# Import PyTorch and related libraries
import torch.nn.functional as F
import torchvision

# Import Scikit-learn utilities
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Display the current working directory
current_path = os.getcwd()
print(f"Current working directory: {current_path}")

# Enable interactive features for Jupyter Notebook
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [4]:
# Don't include the defalut specific token of fastai, only keep the padding token
BOS,EOS,FLD,UNK,PAD = 'xxbos','xxeos','xxfld','xxunk','xxpad'
TK_MAJ,TK_UP,TK_REP,TK_WREP = 'xxmaj','xxup','xxrep','xxwrep'
defaults.text_spec_tok = [PAD]

special_tokens = ['[BOS]', '[C@H]', '[C@@H]','[C@]', '[C@@]','[C-]','[C+]', '[c-]', '[c+]','[cH-]',
                   '[nH]', '[N+]', '[N-]', '[n+]', '[n-]' '[NH+]', '[NH2+]',
                   '[O-]', '[S+]', '[s+]', '[S-]', '[O+]', '[SH]', '[B-]','[BH2-]', '[BH3-]','[b-]',
                   '[PH]','[P+]', '[I+]',
                  '[Si]','[SiH2]', '[Se]','[SeH]', '[se]', '[Se+]', '[se+]','[te]','[te+]', '[Te]',
                  '[Pd+2]', '[Cs+]','[N@@+]','[Na+]','[OH-]','[N@]','[K+]','[F-]','[Rh]','[Ag+]','[Si]', 
                  '[Pd]', '[Cs2+]', '[Cu]', '[Cu+2]', '[Ge]', '[Sb-]', '[Cl+]', '[Cl-]', '[Br-]', '[NH4+]', 
                  '[P-]',
                  ]

class MolTokenizer(BaseTokenizer):
    def __init__(self, lang = 'en', special_tokens = special_tokens):
        self.lang = lang
        self.special_tokens = special_tokens

    def tokenizer(self, smiles):
        # add specific token '[BOS]' to represetences the start of SMILES
        smiles = '[BOS]' + smiles
        regex = '(\[[^\[\]]{1,10}\])'
        char_list = re.split(regex, smiles)
        tokens = []

        if self.special_tokens:
            for char in char_list:
                if char.startswith('['):
                    if char in special_tokens:
                        tokens.append(str(char))
                    else:
                        tokens.append('[UNK]')
                else:
                    chars = [unit for unit in char]
                    [tokens.append(i) for i in chars]

        if not self.special_tokens:
            for char in char_list:
                if char.startswith('['):
                    tokens.append(str(char))
                else:
                    chars = [unit for unit in char]
                    [tokens.append(i) for i in chars]

        #fix the 'Br' be splited into 'B' and 'r'
        if 'B' in tokens:
            for index, tok in enumerate(tokens):
                if tok == 'B':
                    if index < len(tokens)-1: # make sure 'B' is not the last character
                        if tokens[index+1] == 'r':
                            tokens[index: index+2] = [reduce(lambda i, j: i + j, tokens[index : index+2])]

        #fix the 'Cl' be splited into 'C' and 'l'
        if 'l' in tokens:
            for index, tok in enumerate(tokens):
                if tok == 'l':
                    if tokens[index-1] == 'C':
                            tokens[index-1: index+1] = [reduce(lambda i, j: i + j, tokens[index-1 : index+1])]
        return tokens

    def add_special_cases(self, toks):
        pass
    


In [5]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [6]:
def randomize_smiles(smiles):
    m = Chem.MolFromSmiles(smiles)
    ans = list(range(m.GetNumAtoms()))
    np.random.shuffle(ans)
    nm = Chem.RenumberAtoms(m,ans)
    return Chem.MolToSmiles(nm, canonical=False, isomericSmiles=True, kekuleSmiles=False)

In [7]:
def ee_smiles_augmentation(df, N_rounds, noise):
    '''
    noise: add gaussion noise to the label
    '''
    dist_aug = {col_name: [] for col_name in df}

    for i in range(df.shape[0]):
        for j in range(N_rounds):
            dist_aug['smiles'].append(randomize_smiles(df.iloc[i].smiles))
            dist_aug['Yield'].append(df.iloc[i]['Yield'] + np.random.normal(0,noise))
    print(len(dist_aug['smiles']))
    print(len(dist_aug['Yield']))
    #print(len(smiles))
    df_aug = pd.DataFrame.from_dict(dist_aug)
    df_aug = df_aug.append(df, ignore_index=True)
    return df_aug.drop_duplicates('smiles')

In [8]:
def test_smiles_augmentation(df, N_rounds):
    dist_aug = {col_name: [] for col_name in df}

    for i in range(df.shape[0]):
        for j in range(N_rounds):
            dist_aug['smiles'].append(randomize_smiles(df.iloc[i].smiles))
            dist_aug['Yield'].append(df.iloc[i]['Yield'])
    df_aug = pd.DataFrame.from_dict(dist_aug)

    return pd.DataFrame.from_dict(dist_aug)

In [19]:
random_seed(1234, True)

# Create a path to save the results
data_path = Path('./ulmfit/results')
name = 'regressor-minor'
path = data_path / name
path.mkdir(exist_ok=True, parents=True)

In [23]:
# Define a function to process a single sheet
def process_sheet(sheet_name):
    df = pd.read_excel('./Data/C-H-activation/regression/mean-std/866-155-0-mean-std-CV1-20.xlsx', sheet_name=sheet_name)
    df = df[['smiles', 'Yield']]
    df['Yield'] = [int(i) for i in df['Yield']]


    df = df[['smiles', 'Yield']]

    
    train = df.iloc[:108, :]
    valid = df.iloc[108:124, :]
    test = df.iloc[124:, :]
    
    random_seed(1234, True)

    train_aug = ee_smiles_augmentation(train, 50, noise=0.3)
    print("Train_aug: ", train_aug.shape)
    

        
    bs = 64
    tok = Tokenizer(partial(MolTokenizer, special_tokens = special_tokens), n_cpus=6, pre_rules=[], post_rules=[])
        
    np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
    random_seed(1234, True)
        
    lm_vocab = TextLMDataBunch.from_df(path, train_aug, valid, bs=bs, tokenizer=tok,
                              chunksize=50000, text_cols=0,label_cols=1, max_vocab=60000, include_bos=False, min_freq=1, num_workers=0)
    print(f'Vocab Size: {len(lm_vocab.vocab.itos)}')
    
    pretrained_model_path = Path('./Pretraining_weights_bias/SSP1/models/')
    pretrained_fnames = ['C-H-activation_100_wt', 'C-H-activation_100_vocab']
    fnames = [pretrained_model_path/f'{fn}.{ext}' for fn,ext in zip(pretrained_fnames, ['pth', 'pkl'])]
        
    random_seed(1234, True)

    data_clas = TextClasDataBunch.from_df(path, train_aug, valid, bs=bs, tokenizer=tok,
                                                  chunksize=50000, text_cols='smiles',label_cols='Yield',
                                                  vocab=lm_vocab.vocab, max_vocab=60000, include_bos=False, min_freq=1, num_workers=0)
        
    print(f'Vocab Size: {len(data_clas.vocab.itos)}')
        
    random_seed(1234, True)

    lm_learner = language_model_learner(lm_vocab, AWD_LSTM, drop_mult=0.5, wd=0, pretrained=False)
    lm_learner = lm_learner.load_pretrained(*fnames)
    lm_learner.freeze()
    lm_learner.save_encoder(f'lm_encoder') 
        
    random_seed(1234, True)

    reg_learner = text_classifier_learner(data_clas, AWD_LSTM, pretrained=False, drop_mult=0.5, wd=0, metrics = [r2_score, rmse])
    reg_learner.load_encoder(f'lm_encoder')
    reg_learner.freeze()  
        
    reg_learner.model
        
    random_seed(1234, True)

    reg_learner.unfreeze()
    reg_learner.fit_one_cycle(20, 0.001, moms=(0.8,0.7))
        
    split_id = 1
    reg_learner.save(f'{split_id}_reg-p3-aug-155')
        
    preds = []

    # Randomized SMILES Predictions
    for i in range(4):
        np.random.seed(i)
        test_aug = test_smiles_augmentation(test,1)
        
        #model
        test_db = TextClasDataBunch.from_df(path, train_aug, test_aug, tokenizer=tok, vocab=lm_vocab.vocab,
                                                    text_cols='smiles', label_cols='Yield', bs=bs, include_bos=False)
        
        learner = text_classifier_learner(test_db, AWD_LSTM, pretrained=False, drop_mult=0.5, wd=0, metrics = [r2_score, root_mean_squared_error])
        #print(test_db)
        learner.load(f'{split_id}_reg-p3-aug-155');
        
        #get predictions
        pred,lbl = learner.get_preds(ds_type=DatasetType.Valid)
        
        preds.append(pred)
        
    # Canonical SMILES Predictions
    test_db = TextClasDataBunch.from_df(path, train_aug, test, bs=bs, tokenizer=tok,
                                      chunksize=50000, text_cols='smiles',label_cols='Yield', vocab=lm_vocab.vocab, max_vocab=60000,
                                                      include_bos=False)
        
    learner = text_classifier_learner(test_db, AWD_LSTM, pretrained=False, drop_mult=0.5, wd=0.0, metrics = [r2_score, root_mean_squared_error])
        
    learner.load(f'{split_id}_reg-p3-aug-155');
        
        
    #get predictions
    pred_canonical,lbl = learner.get_preds(ds_type=DatasetType.Valid)
        
    preds.append(pred_canonical)
    
    print('Test Set (Canonical)')
    print('RMSE:', root_mean_squared_error(pred_canonical,lbl))
    print('MAE:', mean_absolute_error(pred_canonical,lbl))
    print('R2:', r2_score(pred_canonical,lbl))
    avg_preds = sum(preds)/len(preds)
    #print('\n')
    print('Test Set (Average)')
    print('RMSE:', root_mean_squared_error(avg_preds,lbl))
    print('R2:', r2_score(avg_preds,lbl))
    print('MAE:', mean_absolute_error(avg_preds,lbl))
    
    pred1=pd.DataFrame(pred)
    def remove_tensor(x):
        return float(str(x).replace('tensor(', '').replace(')', ''))
    pred1 = pred1.applymap(remove_tensor)
    print(pred1)
        
    # Concatenate the two arrays column-wise
    concatenated_arr = np.column_stack((test, pred1))
        
    # Print the result
    print(concatenated_arr)
        
    a1=pd.DataFrame(concatenated_arr)
   
        
# Loop through all sheet names and process each sheet
for sheet_num in range(1, 21):  # Assuming sheet names are FullCV_1, FullCV_2, ..., FullCV_20
    sheet_name = f'FullCV_{sheet_num}'
    print(f"Processing sheet: {sheet_name}")
    process_sheet(sheet_name)
        

Processing sheet: FullCV_1
5400
5400
Train_aug:  (5508, 2)


Vocab Size: 48


Vocab Size: 48


epoch,train_loss,valid_loss,r2_score,root_mean_squared_error,time
0,2248.280029,2281.782715,-65.086250,47.768009,00:04
1,2248.071045,2279.178223,-65.010818,47.740738,00:04
2,2227.215088,2286.259277,-65.215904,47.814842,00:04
3,2179.624512,2168.209961,-61.796894,46.564041,00:04
4,2029.011963,2064.995361,-58.807537,45.442219,00:04
5,1797.541382,1952.026367,-55.535667,44.181744,00:04
6,1480.142212,1562.709839,-44.260067,39.531124,00:04
7,1151.659546,1138.579224,-31.976162,33.742840,00:04
8,854.160950,676.787231,-18.601486,26.015135,00:04
9,577.799683,615.012207,-16.812323,24.799440,00:04


Test Set (Canonical)
RMSE: tensor(6.3736)
MAE: tensor(5.8181)
R2: tensor(0.0037)
Test Set (Average)
RMSE: tensor(5.8562)
R2: tensor(0.1589)
MAE: tensor(4.8480)
          0
0   46.0592
1   46.7409
2   49.7866
3   42.8919
4   29.6484
5   41.6454
6   50.6525
7   49.2192
8   42.6454
9   47.2393
10  27.0986
11  45.5631
12  43.5051
13  46.9712
14  47.3269
15  39.2149
16  36.7612
17  45.7785
18  45.6886
19  40.4670
20  46.4093
21  48.6937
22  49.8427
23  42.1161
24  51.6504
25  42.9580
26  45.7347
27  43.1150
28  46.5649
29  49.2936
30  44.5411
[['O=S(CCC1=CC=CC=C1)(OC2=C(C#N)C=CC=C2)=O.C=CC(N(C)C)=O.CC(=O)[O-].CC(=O)[O-].[Pd+2].C(C(=O)O)NC=O.CC(=O)[O-].[Ag+].C(C(F)(F)F)(C(F)(F)F)O'
  51 46.0592]
 ['O=C(N(C1=CC=CC=C1C#N)C2=CC=CC=C2C#N)C(C)(C)OC3=C(C)C=CC=C3.C=CC1=CC=C(OC(C)=O)C=C1.CC(=O)[O-].CC(=O)[O-].[Pd+2].CC(=O)NCC(=O)O.CC(=O)[O-].[Ag+].OC(C(F)(F)F)C(F)(F)F'
  31 46.7409]
 ['O=C(C(F)OC1=C(C#N)C=CC=C1)N(CCC2)C3=C2C=CC=C3C.C=CC(OCC)=O.CC(=O)[O-].CC(=O)[O-].[Pd+2].CC(=O)NCC(=O)O.CC(=O)[O-].[

...

Test Set (Canonical)
RMSE: tensor(7.3461)
MAE: tensor(5.9428)
R2: tensor(-0.9152)
Test Set (Average)
RMSE: tensor(6.8123)
R2: tensor(-0.6470)
MAE: tensor(5.4742)
          0
0   44.7770
1   43.9153
2   43.5340
3   40.9190
4   49.8036
5   43.7999
6   49.1266
7   40.2749
8   45.1634
9   46.1854
10  44.2239
11  41.2151
12  43.8230
13  46.4873
14  25.4660
15  44.7294
16  44.0760
17  41.6746
18  50.8037
19  38.5317
20  49.5227
21  39.0129
22  46.5590
23  44.0003
24  48.8919
25  43.5285
26  49.1066
27  50.9152
28  45.8241
29  49.7728
30  43.5187
[['O=S(CC1=C(Cl)C=C(Cl)C=C1)(OC2=C(C#N)C=CC(OC)=C2)=O.C=CC(OCC)=O.Cl[Rh].Cl[Rh].C1CC=CCCC=C1.C2CC=CCCC=C2.CC(C1=C(C2=C(P(C3CCCCC3)C4CCCCC4)C=CC=C2)C(C(C)C)=CC(C(C)C)=C1)C.O.FC(F)(F)C(=O)O[Cu]OC(=O)C(F)(F)F.O=[V](=O)O[V](=O)=O.ClCCCl'
  51 44.777]
 ['O=S(C1=CC(C(CC(C)C)(C#N)CC(C)C)=C(OC)C=C1)(N2[C@@](CCCC3)([H])[C@@]3(C)C4=C2C=C([H])C=C4)=O.C=CC1=CC=C(C(OC)=O)C=C1.CC(=O)[O-].CC(=O)[O-].[Pd+2].CC(=O)NCC(=O)O.CC(=O)[O-].[Ag+].C(C(F)(F)F)(C(F)(F)F)O'
  4

Vocab Size: 48


Vocab Size: 48


epoch,train_loss,valid_loss,r2_score,root_mean_squared_error,time
0,2235.599854,2287.193359,-29.002127,47.824612,00:04
1,2251.029053,2298.316895,-29.148039,47.940765,00:04
2,2233.446045,2210.602051,-27.997444,47.017040,00:04
3,2187.971924,2192.181641,-27.755816,46.820740,00:04
4,2038.961060,1870.133911,-23.531374,43.245045,00:04
5,1792.936523,1615.648804,-20.193180,40.195133,00:04
6,1480.723267,1204.192017,-14.795919,34.701469,00:04
7,1155.045776,940.716187,-11.339790,30.671097,00:04
8,849.731934,733.620850,-8.623229,27.085436,00:04
9,580.922791,622.116943,-7.160583,24.942272,00:04


Test Set (Canonical)
RMSE: tensor(7.2037)
MAE: tensor(5.9740)
R2: tensor(-0.3454)
Test Set (Average)
RMSE: tensor(6.8779)
R2: tensor(-0.2264)
MAE: tensor(5.4095)
          0
0   50.9478
1   48.1792
2   49.9903
3   29.5540
4   51.2948
5   47.3259
6   42.7528
7   35.2597
8   41.3117
9   41.6610
10  40.8702
11  41.1073
12  46.6863
13  41.8093
14  44.0882
15  39.9553
16  46.7847
17  46.5384
18  50.1055
19  48.7514
20  48.9665
21  38.0222
22  42.2332
23  52.3152
24  40.4010
25  44.8113
26  44.0978
27  40.9627
28  40.3246
29  48.2956
30  46.5508
[['O=C(OC1=C(C#N)C=CC=C1)CC2=CC(Cl)=CC=C2.C=CC(OCCCC)=O.CC(=O)[O-].CC(=O)[O-].[Pd+2].CC(=O)NCC(=O)O.C(=O)([O-])[O-].[Ag+].[Ag+].OC(C(F)(F)F)C(F)(F)F'
  49 50.9478]
 ['CCC(CC)(C#N)C1=CC=CC(C[Si](C(C)C)(C(C)C)OC2=CC([H])=CC(C(F)(F)F)=C2)=C1.C=CC(OCC)=O.Cl[Rh]Cl.Cl[Rh]Cl.C[C]1[C](C)[C](C)[C](C)[C]1C.C[C]2[C](C)[C](C)[C](C)[C]2C.O.FC(F)(F)C(=O)O[Cu]OC(=O)C(F)(F)F.O=[V](=O)O[V](=O)=O.C(CCl)Cl'
  50 48.1792]
 ['O=C(N(C1=C(C#N)C=CC=C1)C2=CC=CC=C2C#N)CC3=CC=